In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
import mxnet as mx
import numpy as np

from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import os
import sys
import math
import time
import pickle

proj_dir = os.getcwd()
label_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/labels')
lld_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/lld')

pickle_train_y_list = 'pickle_train_y_list'
pickle_train_x_list = 'pickle_train_x_list'

pickle_test_y_list = 'pickle_test_y_list'
pickle_test_x_list = 'pickle_test_x_list'

In [ ]:
mfcc_dir = os.path.join(proj_dir, 'emotiondetection/mfcc_csv')

pickle_train_mfcc_x_list = 'pickle_train_mfcc_x_list'

pickle_test_mfcc_x_list = 'pickle_test_mfcc_x_list'

In [ ]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))


train_mfcc_x_list = pickle.load(open(pickle_train_mfcc_x_list))

test_mfcc_x_list = pickle.load(open(pickle_test_mfcc_x_list))

In [ ]:
def calc_score(test_y_list, predict_y_list):
    print "Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):"
    print 'macro   : ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='macro')
    print 'weighted: ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='weighted')

In [ ]:
# z-score normalization
# scaler = preprocessing.StandardScaler().fit(train_x_list)
# train_x_list = scaler.transform(train_x_list)
# test_x_list = scaler.transform(test_x_list)

In [ ]:
# PCA whiten
# pca = PCA(n_components='mle', svd_solver='full', whiten=True)
# x_pca = pca.fit(train_x_list)
# train_x_list = pca.transform(train_x_list)
# test_x_list = pca.transform(test_x_list)

In [ ]:
# sklearn l1/l2 normalization
# x_normalizer = preprocessing.Normalizer(norm='l2').fit(train_x_list)
# train_x_list = x_normalizer.transform(train_x_list)
# test_x_list = x_normalizer.transform(test_x_list)

In [ ]:
# # SMOTE up-sampling training data
# sm = SMOTE(kind='regular')
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)

# # split training data to 5 different classes
# train_x_lists = [[] for i in range(5)]
# for i in xrange(len(train_y_list)):
#     train_x_lists[int(train_y_list[i])].append(train_x_list[i])
# for i in xrange(5):
#     print 'number of class', i, ':', len(train_x_lists[i])

In [ ]:
print train_y_list.shape, train_x_list.shape
print test_y_list.shape, test_x_list.shape

In [ ]:
print train_x_list[0][:10]
print test_x_list[0][:10]

In [ ]:
X = train_x_list
y = train_y_list

In [ ]:
from hmmlearn.hmm import GaussianHMM
from hmmlearn.hmm import GMMHMM
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
# split training data to 5 different classes
train_mfcc_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list)):
    train_mfcc_x_lists[int(train_y_list[i])].append(train_mfcc_x_list[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_mfcc_x_lists[i])

In [ ]:
print len(train_mfcc_x_lists[0])
print train_mfcc_x_lists[0][0].shape
print train_mfcc_x_lists[0][1].shape

In [ ]:
# n_mix_list = [1, 3]
# n_components_list = [32, 512]

n_mix_list = [1]
n_components_list = [32]

for n1 in n_mix_list:
    for n2 in n_components_list:
        print '(GMMHMM) ', 'n_mix:', n1, ';',  'n_components:', n2
        
        # train models and get scores
        models = []
        scores = []
        for i in range(5):
            list_X = train_mfcc_x_lists[i]
            X = np.concatenate(list_X)
            X_lengths = np.array([len(tmp) for tmp in list_X])
            model = GMMHMM(n_mix=n1, n_components=n2).fit(X, X_lengths)
            models.append(model)

            score = []
            for XX in test_mfcc_x_list:
                score.append(model.score(np.array(XX), [len(XX)]))
            scores.append(score)

        predict_y_list = np.argmax(np.array(scores).T, axis = 1)
        calc_score(test_y_list, predict_y_list.tolist())
        
        if not os.path.exists('results') or not os.path.isdir('results'):
            os.path.mkdir('results')
        save_result_path = 'results/gmmhmm_' + str(n1) + '_' + str(n2) + '_predict_y_list.pickle'
        pickle.dump(predict_y_list, open(save_result_path, 'w'))
        
        print '-----------------------'

In [ ]:
print predict_y_list[:100]

In [ ]:
print test_y_list[:100]